## HotChili Analytics trading notebook template
#### Configure by setting ALGO_NAME in cell below.
#### Run various options (backtest, ingest, live) by uncommenting one cell 

In [1]:
%matplotlib inline
%load_ext zipline

# %reload_ext zipline # Uncomment and use this when already loaded zipline extension for magic cell usage.

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


In [2]:
import pandas as pd

# Options you can uncomment and set:

# pd.set_option("max_colwidth", 300)
# pd.set_option("display.max_rows", 300)
# pd.set_option("display.max_columns", 50)
# pd.set_option('precision', 2)
# pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
import os

hca_root_path = os.environ['HCA_ROOT']
print(f"hca_root_path = {hca_root_path}")

hca_root_path = /home/hca-ws2004/hca


# Construct algorithm strategy path names

Assumptions:

- the strategy is in a directory with the same name as the strategy in `ALGO_NAME` below
- the strategy is located in the hca-resources directory, which is located relative to `hca_root_path`, found above

In [4]:
ALGO_NAME = "HCA_Fundamentals" # <--- Supply name here

Other variables are derived from `ALGO_NAME`:

In [5]:
HCA_RESOURCES_PATH = hca_root_path + "/hca-resources/" 
ALGO_PATH          = HCA_RESOURCES_PATH + ALGO_NAME + "/" 
ALGO_BT            = ALGO_PATH + ALGO_NAME + "_Live" + ".py"
ALGO_BT_OUT        = ALGO_PATH + ALGO_NAME + ".pkl"
ALGO_LIVE          = ALGO_PATH + ALGO_NAME + "_Live" + ".py"

print(f"""
ALGO_NAME          = {ALGO_NAME}
HCA_RESOURCES_PATH = {HCA_RESOURCES_PATH}
ALGO_PATH          = {ALGO_PATH}
ALGO_BT            = {ALGO_BT}
ALGO_BT_OUT        = {ALGO_BT_OUT}
ALGO_LIVE          = {ALGO_LIVE}

Contents of algo directory:
""")

!ls $ALGO_PATH


ALGO_NAME          = HCA_Fundamentals
HCA_RESOURCES_PATH = /home/hca-ws2004/hca/hca-resources/
ALGO_PATH          = /home/hca-ws2004/hca/hca-resources/HCA_Fundamentals/
ALGO_BT            = /home/hca-ws2004/hca/hca-resources/HCA_Fundamentals/HCA_Fundamentals_Live.py
ALGO_BT_OUT        = /home/hca-ws2004/hca/hca-resources/HCA_Fundamentals/HCA_Fundamentals.pkl
ALGO_LIVE          = /home/hca-ws2004/hca/hca-resources/HCA_Fundamentals/HCA_Fundamentals_Live.py

Contents of algo directory:

Fundy_sharadar_test.ipynb  HCA_Fundamentals.pkl		     strategy.state
HCA_Fundamentals.ipynb	   HCA_Fundamentals_tearsheet.ipynb
HCA_Fundamentals_Live.py   realtime-bars


## Zipline backtest: 

- Method: Jupyter magic cell (%%) 
- Execution of zipline code in cell containing command line command
- Uncomment first line and hit (shift-enter) inside the cell to run simulation backtest

## Display your current bundles

In [6]:
#!zipline bundles # Finds all bundles

## Ingest Bundle funds assets. 

In [7]:
# Ingest Sharadar funds assets for today, if needed.

# Only need to ingest Funds for this algo, and this takes less processing time and system memory than ingesting
# all of Sharadar Equities plus Funds bundle (sharadar-eqfd)

# Yahoo Direct Data Bundle Ingest
# Quick way
# !export YAHOO_SYM_LST=SPY,ZSL,KOLD,GLD,SHY,XLP,IEF,TLT,QQQ;zipline ingest -b yahoo_direct

# Long way - Set environ variable, so can use it later after bundle is created to find out what is in the bundle.
#os.environ["YAHOO_SYM_LST"] = "SPY,ZSL,KOLD,GLD,SHY,XLP,IEF,TLT,QQQ"
#!zipline ingest -b yahoo_direct
#yahoo_direct_sym_lst = os.environ['YAHOO_SYM_LST']
#print(f"yahoo_direct_sym_lst = {yahoo_direct_sym_lst}")

#!zipline ingest -b sharadar-funds 
#!zipline ingest -b sharadar-prices 
##### !zipline ingest -b hca-symbol


## Zipline backtest, alternative method

- Method: command line
- Execution of zipline code, located in a file, using below command line execution with magic (`!`) invocation
- This line can also be run in a terminal by copying everything past the `!` and pasting (shift-insert) it into the target terminal

In [8]:
#!zipline run -f $ALGO_LIVE  --start=2019-04-01 --end=2021-04-15 -b sharadar-eqfd -o $ALGO_BT_OUT

## Run Zipline live on IB-TWS via command line

- Method: command line
- Execution of zipline code using below command line execution using magic (`!`) invocation

**Notes:** 
- IB-TWS or IB-Gateway must be running, with `IB_ACCT` and `IB_URI` port being correct to live trade
- Change `I_WANT_TO_RUN_THIS_CODE` to `True` below to run zipline live on IB-TWS/IB-Gateway

In [9]:
TODAY = pd.datetime.today().strftime("%Y-%m-%d")
print("TODAY = {}".format(TODAY))

TODAY = 2021-04-20


In [10]:
ALGO_STATE = ALGO_PATH + "strategy.state" 
ALGO_RTB   = ALGO_PATH + "realtime-bars/"

# Edit the following URI to match your IB account and port info.
IB_ACCT = "DU1568488"
IB_URI = "127.0.0.1:7497:1301"
BUNDLE = 'sharadar-eqfd'
#BUNDLE = 'yahoo_direct'
#BUNDLE = 'hca_symbol'
!echo zipline run \
        -s $TODAY \
        -f $ALGO_LIVE \
        --bundle $BUNDLE \
        --broker ib \
        --broker-uri $IB_URI \
        --broker-acct $IB_ACCT \
        --data-frequency daily \
        --state-file $ALGO_STATE \
        --realtime-bar-target $ALGO_RTB 
# Change following to 'True' and run cell (control-enter) to execute live run.
#I_WANT_TO_RUN_THIS_CODE = False
I_WANT_TO_RUN_THIS_CODE = True

if I_WANT_TO_RUN_THIS_CODE:    
    !zipline run \
        -s $TODAY \
        -f $ALGO_LIVE \
        --bundle $BUNDLE \
        --broker ib \
        --broker-uri $IB_URI \
        --broker-acct $IB_ACCT \
        --data-frequency daily \
        --state-file $ALGO_STATE \
        --realtime-bar-target $ALGO_RTB 

zipline run -s 2021-04-20 -f /home/hca-ws2004/hca/hca-resources/HCA_Fundamentals/HCA_Fundamentals_Live.py --bundle sharadar-eqfd --broker ib --broker-uri 127.0.0.1:7497:1301 --broker-acct DU1568488 --data-frequency daily --state-file /home/hca-ws2004/hca/hca-resources/HCA_Fundamentals/strategy.state --realtime-bar-target /home/hca-ws2004/hca/hca-resources/HCA_Fundamentals/realtime-bars/
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.Na